In [1]:
import bpy
# import os
import math
from pathlib import Path

In [2]:
# Function to ensure there is an active camera in the scene
def ensure_camera_exists():
    # Check if there is any camera in the scene
    cameras = [obj for obj in bpy.data.objects if obj.type == 'CAMERA']
    if cameras:
        # Set the first found camera as the active camera
        bpy.context.scene.camera = cameras[0]
    else:
        # Create and set up the camera
        bpy.ops.object.camera_add(location=(12.391, -1412.3, 457.6))
        camera = bpy.context.object
        camera.data.type = 'PERSP'
        camera.data.clip_end = 100000
        camera.rotation_mode = 'XYZ'
        camera.rotation_euler[0] = math.radians(76.4)
        camera.rotation_euler[1] = math.radians(-0.000029)
        camera.rotation_euler[2] = math.radians(-0.000001)
        bpy.context.scene.camera = camera

# Function to update the scene and context
def update_scene():
    bpy.context.view_layer.update()
    bpy.context.evaluated_depsgraph_get().update()

# Function to render and save the image in the specified directory structure
def render_and_save(frame_number, track_name, strip_name, champion_name, skin_name):
    ensure_camera_exists()
    bpy.context.scene.frame_set(frame_number)
    # Adjust the filename to include the strip name for uniqueness
    filename = f"{track_name}_{strip_name}_frame{frame_number}.png"
    # Create directory path for the current champion and skin
    champion_skin_dir = Path(output_dir) / champion_name / skin_name
    champion_skin_dir.mkdir(parents=True, exist_ok=True)
    file_path = champion_skin_dir / filename
    bpy.context.scene.render.filepath = str(file_path)
    bpy.ops.render.render(write_still=True)


# Function to clear all objects in the scene
def clear_scene():
    # Select and delete all objects
    bpy.ops.object.select_all(action='SELECT')
    bpy.ops.object.delete()
    # Purge orphan data
    bpy.ops.outliner.orphans_purge()

# Function to add a camera at a specific location and rotation
def add_camera(location, rotation_euler_degrees, name):
    bpy.ops.object.camera_add(location=location)
    camera = bpy.context.object
    camera.data.type = 'PERSP'
    camera.data.clip_end = 100000
    camera.rotation_mode = 'XYZ'
    # Convert degrees to radians and apply rotation
    camera.rotation_euler = [math.radians(angle) for angle in rotation_euler_degrees]
    camera.name = name  # Set the name of the camera
    return camera

# Function to set up multiple cameras
def setup_cameras():
    # ((50, -1300, 400), (80, 0, 15), 'Camera2'),
    # ((-30, -1250, 420), (75, 0, -15), 'Camera3')
    # Define test locations and rotations for cameras
    camera_settings = [
        ((12.391, -1412.3, 457.6), (76.4, 0, 0), 'Camera1'),

    ]

    # Add cameras to the scene
    for location, rotation, name in camera_settings:
        add_camera(location, rotation, name)

# Function to find objects with animation data
def find_objects_with_animation():
    for obj in bpy.data.objects:
        if obj.animation_data and obj.animation_data.action:
            return obj.name
    return None

def find_untracked_actions(armature_name):
    armature = bpy.data.objects.get(armature_name)
    untracked_actions = []

    # Include the current active action if it exists
    if armature.animation_data and armature.animation_data.action:
        untracked_actions.append(armature.animation_data.action.name)

    # Include all linked actions
    for action in bpy.data.actions:
        if action.users and not action.use_fake_user:
            # Check if the action is not already in an NLA track
            if not any(strip.action == action for track in armature.animation_data.nla_tracks for strip in track.strips):
                untracked_actions.append(action.name)

    return list(set(untracked_actions))  # Return unique action names

# Function to move an action to NLA tracks
def move_action_to_nla(armature_name, action_name):
    armature = bpy.data.objects.get(armature_name)
    if armature.animation_data and armature.animation_data.action:
        action = armature.animation_data.action
        if action.name == action_name:
            if not armature.animation_data.nla_tracks:
                armature.animation_data_create()
            
            new_track = armature.animation_data.nla_tracks.new()
            new_track.name = action.name
            new_strip = new_track.strips.new(action.name, int(action.frame_range[0]), action)
            new_strip.action = action
            armature.animation_data.action = None
            print(f"Moved action '{action_name}' into a new NLA track.")
        else:
            print(f"The active action is not '{action_name}', it's '{action.name}'.")
    else:
        print(f"No active action on armature '{armature_name}' to move.")

def process_all_tracks(armature_name, champion_name, skin_name, track_limit=None):
    armature = bpy.data.objects.get(armature_name)
    bpy.context.view_layer.objects.active = armature
    armature.select_set(True)

    if armature.animation_data:
        # First mute all tracks
        for track in armature.animation_data.nla_tracks:
            track.mute = True
        tracks_processed = 0
        for track in armature.animation_data.nla_tracks:
            if track_limit is not None and tracks_processed >= track_limit:
                break  # Stop processing if the limit is reached

            track.mute = False
            update_scene()  # Update scene after unmuting the track

            for strip in track.strips:
                frame_start = int(strip.frame_start)
                frame_end = int(strip.frame_end)
                animation_length = frame_end - frame_start

                # Adjusted dynamic interval calculation
                if animation_length <= 20:
                    interval = 2
                elif animation_length <= 100:
                    interval = max(2, animation_length // 20)
                else:
                    interval = max(5, animation_length // 50)

                # Render at calculated intervals within this strip
                for frame in range(frame_start, frame_end + 1, interval):
                    render_and_save(frame, track.name, strip.name, champion_name, skin_name)

            track.mute = True
            update_scene()

            tracks_processed += 1  # Increment the counter after processing a track


# Function to get the base directory of the league_object_detection_tracking folder
def get_base_directory():
    # Use the Path class to get the current working directory
    return Path.cwd().parent

# Function to disable backface culling for all materials of a given object
def disable_backface_culling(obj):
    if obj.type == 'MESH':
        for mat_slot in obj.material_slots:
            if mat_slot.material:
                # Directly set use_backface_culling on the material, not the shader node
                mat_slot.material.use_backface_culling = False

def log_failed_imports(champion_name, skin_name, log_file='failed_imports.txt'):
    """
    Logs the names of champions and skins that failed to import to a text file.
    
    Args:
    champion_name (str): Name of the champion.
    skin_name (str): Name of the skin.
    log_file (str): Path to the log file.
    """
    with open(log_file, 'a') as file:  # 'a' mode appends to the file without overwriting existing data
        file.write(f"{champion_name} - {skin_name}\n")

In [3]:
# Base directory for the league_object_detection_tracking folder
base_directory = get_base_directory()
# print(f"base_directory: {base_directory}")

# Paths to the 'models' and 'images' directories
models_directory = base_directory / 'models'

output_dir = base_directory / 'images'
# Print the models directory path for debugging
# print(f"Models Directory: {models_directory}")

# Ensure output directory exists
output_dir.mkdir(parents=True, exist_ok=True)

# Set a limit for the number of models to process
models_limit = 18
# Counter for models proccessed in loop
models_processed = 0
# Set a limit for the number of Tracks to process
track_limit = 2
# List of champions to process, leave empty to process all champions
champions_to_process = ['Ahri']  # Add champion names like 'Ahri', 'Aatrox', etc.

# Loop through each champion directory in the models directory
for champion_dir in models_directory.iterdir():
    if models_processed >= models_limit:
        break  # Stop processing if the limit is reached
    if champion_dir.is_dir() and (not champions_to_process or champion_dir.name in champions_to_process):  # Ensure it is a directory
        champion_name = champion_dir.name  # Get the champion name
        # Loop through each skin directory within the champion's directory
        for skin_dir in champion_dir.iterdir():
            if skin_dir.is_dir():  # Ensure it is a directory
                glb_file = skin_dir / f"{skin_dir.name}.glb"  # Construct the file path for the .glb file
                skin_name = skin_dir.name  # Get the skin name
                if glb_file.is_file():  # Check if the .glb file exists
                    # Clear the scene for each new model import
                    clear_scene()
                    
                    # Set up multiple cameras
                    setup_cameras()

                    # Try to import the GLB file
                    try:
                        bpy.ops.import_scene.gltf(filepath=str(glb_file))
                    except RuntimeError as e:
                        print(f"Failed to import {glb_file}: {e}")
                        log_failed_imports(champion_name, skin_name)
                        continue  # Skip to the next file

                    # Disable backface culling for all objects in the scene
                    for obj in bpy.data.objects:
                        disable_backface_culling(obj)

                    # Find the name of the object with animation data
                    animated_object_name = find_objects_with_animation()
                    if animated_object_name:
                        print(f"Found animated object: {animated_object_name}")

                        # Find untracked actions for the animated object
                        untracked_actions = find_untracked_actions(animated_object_name)
                        print(f"Untracked actions for '{animated_object_name}': {untracked_actions}")

                        # Move untracked actions to NLA tracks
                        for action_name in untracked_actions:
                            move_action_to_nla(animated_object_name, action_name)

                        # Process all NLA tracks for the animated object
                        process_all_tracks(animated_object_name, champion_name, skin_name, track_limit)
                        
                    else:
                        print("No animated objects found in this model.")
                models_processed += 1  # Increment the counter after processing a model
                if models_processed >= models_limit:
                    break  # Stop processing if the limit is reached

Info: Deleted 4 data-block(s)
Data are loaded, start creating Blender stuff
glTF import finished in 5.41s
Found animated object: Armature
Untracked actions for 'Armature': ['Attack1_Armature']
Moved action 'Attack1_Armature' into a new NLA track.
Info: Deleted 56 data-block(s)
Data are loaded, start creating Blender stuff
glTF import finished in 5.37s
Found animated object: Armature
Untracked actions for 'Armature': ['Attack1_Armature']
Moved action 'Attack1_Armature' into a new NLA track.
Info: Deleted 57 data-block(s)
Data are loaded, start creating Blender stuff
glTF import finished in 5.49s
Found animated object: Armature
Untracked actions for 'Armature': ['Attack1_Armature']
Moved action 'Attack1_Armature' into a new NLA track.
Info: Deleted 59 data-block(s)
Data are loaded, start creating Blender stuff
glTF import finished in 5.74s
Found animated object: Armature
Untracked actions for 'Armature': ['Attack1_Armature']
Moved action 'Attack1_Armature' into a new NLA track.
Info: Del